<a href="https://colab.research.google.com/github/Linoh12/LearningLSTM/blob/main/LSTM_Lincoln.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import

In [74]:
!pip install scikit-learn-intelex

In [75]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torchvision

import sklearn
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from sklearnex import patch_sklearn
patch_sklearn()

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from pathlib import Path

torch.__version__

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


'2.0.1+cu118'

In [76]:
fp = 'raw_data.csv'
data = pd.read_csv(fp)
data

,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity
0,1,2,-3.000000e-08,0.000006,72.6660,1.400000e-07,42.107007,-3.000000e-08,-3.000000e-08,-4.000000e-08
1,1,3,-3.000000e-08,0.000006,71.0602,1.170000e-07,35.000000,-3.000000e-08,-3.000000e-08,-3.000000e-08
2,1,4,-3.000000e-08,0.000006,73.6697,1.580000e-07,47.265209,-3.000000e-08,-3.000000e-08,-3.000000e-08
3,1,5,-3.000000e-08,0.000006,71.8491,1.280000e-07,38.327536,-3.000000e-08,-3.000000e-08,-3.000000e-08
4,1,6,-3.000000e-08,0.000006,75.8679,2.030000e-07,60.876925,-3.000000e-08,-3.000000e-08,-4.000000e-08
...,...,...,...,...,...,...,...,...,...,...
49995,14,17,-4.959106e-01,81.240209,69.7883,1.010000e-07,30.232433,-4.959106e-01,2.479553e+00,2.479553e+00
49996,14,18,-4.959106e-01,81.240209,65.4033,6.100000e-08,18.248288,-4.959106e-01,2.479553e+00,2.479553e+00
49997,14,19,-5.000000e-08,81.240209,90.7695,1.128000e-06,338.479901,-4.959106e-01,2.479553e+00,-9.000000e-08
49998,14,20,-5.000000e-08,81.240209,90.0503,1.039000e-06,311.579745,-4.959106e-01,2.479553e+00,-9.000000e-08


In [77]:
pairs = [{row[0],row[1]} for row in data[['SenderID','ReceiverID']].values]
# len(np.unique(pairs, axis=1))
pairs

[{1, 2},
 {1, 3},
 {1, 4},
 {1, 5},
 {1, 6},
 {1, 7},
 {1, 8},
 {1, 9},
 {1, 10},
 {1, 11},
 {1, 12},
 {1, 13},
 {1, 14},
 {1, 15},
 {1, 16},
 {1, 17},
 {1, 18},
 {1, 19},
 {1, 20},
 {1, 21},
 {1, 22},
 {1, 23},
 {1, 24},
 {1, 25},
 {1, 26},
 {1, 4},
 {2, 4},
 {3, 4},
 {4, 5},
 {4, 6},
 {4, 7},
 {4, 8},
 {4, 9},
 {4, 10},
 {4, 11},
 {4, 12},
 {4, 13},
 {4, 14},
 {4, 15},
 {4, 16},
 {4, 17},
 {4, 18},
 {4, 19},
 {4, 20},
 {4, 21},
 {4, 22},
 {4, 23},
 {4, 24},
 {4, 25},
 {4, 26},
 {1, 25},
 {2, 25},
 {3, 25},
 {4, 25},
 {5, 25},
 {6, 25},
 {7, 25},
 {8, 25},
 {9, 25},
 {10, 25},
 {11, 25},
 {12, 25},
 {13, 25},
 {14, 25},
 {15, 25},
 {16, 25},
 {17, 25},
 {18, 25},
 {19, 25},
 {20, 25},
 {21, 25},
 {22, 25},
 {23, 25},
 {24, 25},
 {25, 26},
 {1, 14},
 {2, 14},
 {3, 14},
 {4, 14},
 {5, 14},
 {6, 14},
 {7, 14},
 {8, 14},
 {9, 14},
 {10, 14},
 {11, 14},
 {12, 14},
 {13, 14},
 {14, 15},
 {14, 16},
 {14, 17},
 {14, 18},
 {14, 19},
 {14, 20},
 {14, 21},
 {14, 22},
 {14, 23},
 {14, 24},
 {14, 

In [78]:
IDs = np.arange(1,27)
possible_pairs = []
for i,v in enumerate(IDs):
  temp = list(IDs)
  temp.pop(i)
  for choose in temp:
    possible_pairs.append({v,choose})
# possible_pairs = pd.Series(possible_pairs, name='pair')

possible_pairs[:5]

[{1, 2}, {1, 3}, {1, 4}, {1, 5}, {1, 6}]

In [79]:
def make_negatives(possible_pairs, data, random_state, negative_only = True):
  """
  possible_pairs is the total permutations possible between SenderID and RecieverID
  data is the dataset for a given timestamp
  random_state sets the random seed using np.random.seed()
  """

  #Random Seed
  np.random.seed(random_state)
  rng = np.random.default_rng()

  #Remapping the data into
  known_pairs = list(map(set,data[['SenderID','ReceiverID']].values))
  len_known_pairs = len(known_pairs)


  #Manually checking if the pair exists
  arr = []
  for i in possible_pairs:
    found = False
    for j in known_pairs:
      if i==j:
        found = True
        break

    if found:
      arr.append(True)
    else:
      arr.append(False)

  arr = np.array(arr)

  #Filtering the leftover pairs and randomly choosing from them an equal number of negatives as positives
  leftover_pairs = pd.DataFrame(possible_pairs, columns=['SenderID','ReceiverID'])[~arr]
  rng.shuffle(leftover_pairs.values)
  chosen = leftover_pairs[:len_known_pairs].reset_index(drop=True)

  #Creating a deep copy and updating the ID pairs to the random leftovers chosen
  copy = data.copy(deep=True).reset_index(drop=True)
  copy["SenderID"], copy["ReceiverID"], copy["present"] = chosen["SenderID"], chosen["ReceiverID"], 0
  # print(copy[['SenderID','ReceiverID','present']], 'Length:', len(copy), len_known_pairs)

  if negative_only:
    return copy

  else:
    results = pd.concat((data, copy), axis=1)
    return results


# Model Building

In [80]:
#Device Agnostic Code
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


## Data Prep

In [82]:
# Applying Standard Scalar to normalize data
scaled_data =pd.DataFrame(StandardScaler().fit_transform(data), columns=data.columns)
scaled_data

,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity
0,-0.895688,-1.334772,-0.415269,-2.109152,-1.272443,-1.232738,-1.232222,-0.575348,-0.725865,-0.550458
1,-0.895688,-1.200659,-0.415269,-2.109152,-1.427426,-1.264487,-1.264923,-0.575348,-0.725865,-0.550458
2,-0.895688,-1.066545,-0.415269,-2.109152,-1.175571,-1.207890,-1.208487,-0.575348,-0.725865,-0.550458
3,-0.895688,-0.932432,-0.415269,-2.109152,-1.351286,-1.249303,-1.249612,-0.575348,-0.725865,-0.550458
4,-0.895688,-0.798318,-0.415269,-2.109152,-0.963413,-1.145772,-1.145856,-0.575348,-0.725865,-0.550458
...,...,...,...,...,...,...,...,...,...,...
49995,0.742546,0.676930,-0.539460,2.064079,-1.550183,-1.286574,-1.286861,-0.681885,0.091776,0.379528
49996,0.742546,0.811043,-0.539460,2.064079,-1.973399,-1.341790,-1.342003,-0.681885,0.091776,0.379528
49997,0.742546,0.945156,-0.415269,2.064079,0.474808,0.131111,0.131483,-0.681885,0.091776,-0.550458
49998,0.742546,1.079270,-0.415269,2.064079,0.405394,0.008254,0.007707,-0.681885,0.091776,-0.550458


In [83]:
timestamps = np.unique(scaled_data['Timestamp'])
timestamps, timestamps.shape

(array([-2.10915175, -2.10761067, -2.1060696 , ...,  2.06099688,
         2.06356538,  2.06407907]),
 (2233,))

In [84]:
train_size = 0.8
train_timestamps = timestamps[:round(len(timestamps)*train_size)]
test_timestamps = timestamps[round(len(timestamps)*train_size): ]
train_timestamps.shape, test_timestamps.shape

((1786,), (447,))

In [85]:
timestamp_train_data = scaled_data[np.isin(scaled_data['Timestamp'], train_timestamps)]
timestamp_train_data['present'] = 1
timestamp_train_data

<ipython-input-85-38253897283d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timestamp_train_data['present'] = 1


,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity,present
0,-0.895688,-1.334772,-0.415269,-2.109152,-1.272443,-1.232738,-1.232222,-0.575348,-0.725865,-0.550458,1
1,-0.895688,-1.200659,-0.415269,-2.109152,-1.427426,-1.264487,-1.264923,-0.575348,-0.725865,-0.550458,1
2,-0.895688,-1.066545,-0.415269,-2.109152,-1.175571,-1.207890,-1.208487,-0.575348,-0.725865,-0.550458,1
3,-0.895688,-0.932432,-0.415269,-2.109152,-1.351286,-1.249303,-1.249612,-0.575348,-0.725865,-0.550458,1
4,-0.895688,-0.798318,-0.415269,-2.109152,-0.963413,-1.145772,-1.145856,-0.575348,-0.725865,-0.550458,1
...,...,...,...,...,...,...,...,...,...,...,...
39803,-0.895688,1.347497,-0.415269,0.925225,0.764216,0.773003,0.773631,1.035886,1.005340,-0.550458,1
39804,-0.895688,1.481610,-0.415269,0.925225,0.713556,0.644625,0.644631,1.035886,1.005340,-0.550458,1
39805,-0.895688,1.615724,-0.415269,0.925225,0.800216,0.869632,0.870132,1.035886,1.005340,-0.550458,1
39806,-0.895688,1.749837,-0.415269,0.925225,0.747799,0.731591,0.730976,1.035886,1.005340,-0.550458,1


In [86]:
for timestamp in train_timestamps:
  current_timestamp = timestamp_train_data[timestamp_train_data['Timestamp'] == timestamp].copy(deep=True).reset_index(drop=True)
  timestamp_train_data = pd.concat((timestamp_train_data, make_negatives(possible_pairs, current_timestamp, random_state=42)), axis=0)

timestamp_train_data

,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity,present
0,-0.895688,-1.334772,-0.415269,-2.109152,-1.272443,-1.232738,-1.232222,-0.575348,-0.725865,-0.550458,1
1,-0.895688,-1.200659,-0.415269,-2.109152,-1.427426,-1.264487,-1.264923,-0.575348,-0.725865,-0.550458,1
2,-0.895688,-1.066545,-0.415269,-2.109152,-1.175571,-1.207890,-1.208487,-0.575348,-0.725865,-0.550458,1
3,-0.895688,-0.932432,-0.415269,-2.109152,-1.351286,-1.249303,-1.249612,-0.575348,-0.725865,-0.550458,1
4,-0.895688,-0.798318,-0.415269,-2.109152,-0.963413,-1.145772,-1.145856,-0.575348,-0.725865,-0.550458,1
...,...,...,...,...,...,...,...,...,...,...,...
20,12.000000,22.000000,-0.415269,0.925225,0.764216,0.773003,0.773631,1.035886,1.005340,-0.550458,0
21,20.000000,13.000000,-0.415269,0.925225,0.713556,0.644625,0.644631,1.035886,1.005340,-0.550458,0
22,6.000000,22.000000,-0.415269,0.925225,0.800216,0.869632,0.870132,1.035886,1.005340,-0.550458,0
23,19.000000,13.000000,-0.415269,0.925225,0.747799,0.731591,0.730976,1.035886,1.005340,-0.550458,0


In [118]:
def pad_input(data, input_size):
    features = np.zeros((1, input_size, data.shape[2]),dtype=float)
    for ii, row in enumerate(data):
        if len(row) != 0:
            features[ii, -len(row):] = np.array(row)[:input_size]
    return features

input_size = 150  # The length that the data will be padded/shortened to

In [119]:
train_data_seq = None
flag = True
# Seperating the data according to the timestamp
for timestamp in train_timestamps:
  current = timestamp_train_data[timestamp_train_data['Timestamp'] == timestamp].values.tolist()
  current = torch.Tensor(current).unsqueeze(dim=0)
  current = pad_input(current, input_size)
  current = torch.Tensor(current).to(device)

  if flag:
    flag = False
    train_data_seq = current
    continue

  train_data_seq = torch.cat((train_data_seq, current), dim=0)

train_data_seq



tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [12.0000, 23.0000, -0.4153,  ..., -0.7259, -0.5505,  0.0000],
         [14.0000, 15.0000, -0.4153,  ..., -0.7259, -0.5505,  0.0000],
         [20.0000,  6.0000, -0.4153,  ..., -0.7259, -0.5505,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 2.0000,  6.0000, -0.4153,  ..., -0.7259, -0.5505,  0.0000],
         [ 8.0000, 24.0000, -0.4153,  ..., -0.7259, -0.5505,  0.0000],
         [ 3.0000, 13.0000, -0.4153,  ..., -0.7259, -0.5505,  0.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0

In [ ]:

# train_data_seq = []
# flag = True
# for timestamp in train_timestamps:
#   current = timestamp_train_data[timestamp_train_data['Timestamp'] == timestamp].values.tolist()


#   current = torch.Tensor(current).unsqueeze(dim=0).to(device)

#   train_data_seq.append(current)

# train_data_seq[:5]


In [120]:
train_data_seq[0].shape

torch.Size([150, 11])

In [89]:
timestamp_test_data = scaled_data[np.isin(scaled_data['Timestamp'], test_timestamps)]
timestamp_test_data['present'] = 1
timestamp_test_data

<ipython-input-89-588a8fc2f218>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timestamp_test_data['present'] = 1


,SenderID,ReceiverID,Receiver_XVelocity,Timestamp,Pathloss,PropDelay,Distance,Sender_XVelocity,Sender_YVelocity,Receiver_YVelocity,present
39808,0.742546,-1.468886,1.462951,0.927279,-0.197696,-0.727506,-0.727710,-0.576049,-0.720483,1.418618,1
39809,0.742546,-1.334772,1.462951,0.927279,-0.343771,-0.839320,-0.839366,-0.576049,-0.720483,1.418618,1
39810,0.742546,-1.200659,1.462951,0.927279,-0.406187,-0.882113,-0.881455,-0.576049,-0.720483,1.418618,1
39811,0.742546,-1.066545,1.462951,0.927279,-0.434967,-0.900058,-0.899837,-0.576049,-0.720483,1.418618,1
39812,0.742546,-0.932432,1.462951,0.927279,-0.341792,-0.837939,-0.837984,-0.576049,-0.720483,1.418618,1
...,...,...,...,...,...,...,...,...,...,...,...
49995,0.742546,0.676930,-0.539460,2.064079,-1.550183,-1.286574,-1.286861,-0.681885,0.091776,0.379528,1
49996,0.742546,0.811043,-0.539460,2.064079,-1.973399,-1.341790,-1.342003,-0.681885,0.091776,0.379528,1
49997,0.742546,0.945156,-0.415269,2.064079,0.474808,0.131111,0.131483,-0.681885,0.091776,-0.550458,1
49998,0.742546,1.079270,-0.415269,2.064079,0.405394,0.008254,0.007707,-0.681885,0.091776,-0.550458,1


In [121]:
test_data_seq = None
flag = True
# Seperating the data according to the timestamp
for timestamp in test_timestamps:
  current = timestamp_test_data[timestamp_test_data['Timestamp'] == timestamp].values.tolist()
  current = torch.Tensor(current).unsqueeze(dim=0)
  current = pad_input(current, input_size)
  current = torch.Tensor(current).to(device)

  if flag:
    flag = False
    test_data_seq = current
    continue

  test_data_seq = torch.cat((test_data_seq, current), dim=0)

test_data_seq

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.7425, -0.6642,  1.4630,  ..., -0.7205,  1.4186,  1.0000],
         [ 0.7425, -0.5301,  1.4630,  ..., -0.7205,  1.4186,  1.0000],
         [ 0.7425, -0.3960,  1.4630,  ..., -0.7205,  1.4186,  1.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.8686, -0.6642,  1.4630,  ..., -0.7205,  1.4186,  1.0000],
         [ 0.8686, -0.5301,  1.4630,  ..., -0.7205,  1.4186,  1.0000],
         [ 0.8686, -0.3960,  1.4630,  ..., -0.7205,  1.4186,  1.0000]],

        [[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0

In [ ]:
# # Seperating the data according to the timestamp
# test_data_seq = []
# flag = True
# for timestamp in test_timestamps:
#   current = torch.Tensor(timestamp_test_data[timestamp_test_data['Timestamp'] == timestamp].values.tolist()).unsqueeze(dim=0).to(device)

#   test_data_seq.append(current)

# test_data_seq[:5]

In [123]:
test_data_seq.shape

torch.Size([447, 150, 11])

In [18]:
class dataset(torch.utils.data.Dataset):
  def __init__(self,x,y):
    self.x = x
    self.y = y

    # self.y = torch.tensor(y,dtype=torch.float32).to(device)

    self.length = self.x.shape[0]

  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length


## Model

In [81]:
#Hyper Parameters
input_size = 10
num_layers = 1
sequence_length = 5

hidden_size = 10
#num_classes = 1

num_epochs = 300 #3000
batch_size = 16
learning_rate = 0.001 #0.00025

In [19]:
torch.manual_seed(42)

In [20]:
class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
    self.fc = nn.Linear(hidden_size,1)

    self.s1 = nn.Sigmoid()


  def forward(self, x):
      out,_ = self.lstm(x) # x -> (batch_size, seq, input_size)
      # out: (batch_size, seq_length, hidden_size) # from rnn
      # out(N, 5, 5)
      out = out[:, -1, :]
      out = self.fc(out)
      out = self.s1(out)
      return out


In [26]:
model = RNN(input_size, hidden_size, num_layers).to(device)

#loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training

In [92]:
#Training Loop V2
model.train()
num_timstamps = len(train_timestamps)
for tsi, time in enumerate(train_timestamps): #tsi is timestamp index, time is the actual value of the timestamp
  # Get data for current timestamp
  current_train_seq = train_data_seq[tsi]

  # Split to training features/ nodes and target ("present")
  X_train, y_train = current_train_seq[:,:,:-1].clone().detach(), current_train_seq[:,:,-1].clone().detach()

  # Creating trainset and train_loader
  trainset = dataset(X_train,y_train)
  train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size)
  n_total_steps = len(train_loader)

  for epoch in range(num_epochs):
    for i, (x_train,y_train) in enumerate(train_loader):
      #---Forward---
      print(x_train.shape)
      # x_train = x_train.reshape(1,sequence_length, -1)
      outputs = model(x_train)
      y_train = y_train[:,-1].unsqueeze(dim=1)

      # print(outputs,y_train,sep='\n')
      # print(outputs.shape, y_train.shape)

      loss = criterion(outputs, y_train)

      #---Backward---
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # Printing progress
      if (tsi+1)%100 == 0:
        if (epoch+1)%100 == 0:
          print(f'Timestamp {tsi+1}/{num_timstamps}, Epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item()}')

      if (epoch+1)==num_epochs:
        print('')

Streaming output truncated to the last 5000 lines.
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1, 50, 10])
torch.Size([1

KeyboardInterrupt: ignored

In [22]:
# #Training Loop
# model.train()

# n_total_steps = len(train_loader)
# for epoch in range(num_epochs):
#   for i, (x_train,y_train) in enumerate(train_loader):
#     # print(f'Iter {i+1}, x_train:{x_train[:5]}, y_train:{y_train[:5]}')

#     #forward
#     # print('x_train before reshaping: ',x_train.shape)
#     x_train = x_train.reshape(-1, sequence_length, input_size).to(device)
#     # print('x_train after reshaping: ', x_train.shape)
#     outputs = model(x_train)

#     y_train = y_train[:,-1].unsqueeze(dim=1)

#     # print(outputs,y_train,sep='\n')
#     # print(outputs.shape, y_train.shape)
#     loss = criterion(outputs, y_train)

#     #backward
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     if (epoch+1)%100 == 0:
#       print(f'Epoch {epoch+1}/{num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.5f}')

#       if (i+1)==n_total_steps:
#         print('')

## Testing

In [ ]:
model.input_size, model.num_layers, model.hidden_size

In [ ]:
def binary_threshold(x, threshold= 0.5):
  # print(x)
  threshold = torch.tensor([threshold]).to(device)
  results = (x>threshold).float()*1
  # print(results)
  return results

In [ ]:
#Testing Loop for Training Data
model.eval()
n_correct = 0
n_samples = 0
with torch.inference_mode():
  for i, (x_train,y_train) in enumerate(train_loader):
    x_train = x_train.reshape(-1, sequence_length, input_size).to(device)
    y_train = y_train[:,-1].unsqueeze(dim=1).to(device)
    outputs = model(x_train)
    outputs = binary_threshold(outputs)

    # print(outputs,y_train)
    # print(outputs.shape, y_train.shape)

    n_samples += y_train.shape[0] * y_train.shape[1]

    n_correct += (outputs == y_train).sum().item()
    # print((outputs==y_train).sum())
    # print(n_correct, n_samples)

acc = 100* n_correct/n_samples
print(f'Training Accuracy = {acc:.3f}%')

In [ ]:
#Testing Loop for Testing Data
model.eval()
n_correct = 0
n_samples = 0
with torch.inference_mode():
  for i, (x_test,y_test) in enumerate(test_loader):
    x_test = x_test.reshape(-1, sequence_length, input_size).to(device)
    y_test = y_test.to(device)
    outputs = model(x_test)
    outputs = binary_threshold(outputs)

    # print(outputs,y_test)
    # print(outputs.shape, y_test.shape)

    ### If we only use the new unseen data
    outputs = outputs[:,-1]
    y_test = y_test[:,-1]

    # print(outputs,y_test)
    # print(outputs.shape, y_test.shape)

    # n_samples += y_test.shape[0] * y_test.shape[1]
    n_samples += y_test.shape[0]

    n_correct += (outputs == y_test).sum().item()

    print(f'Cumulative: {n_correct}/{n_samples}')

acc = 100* n_correct/n_samples
print(f'Testing Accuracy = {acc:.3f}%')